# Automated Road Damage Classification System
## deep learning project with Computer Vision & Grad-CAM Visualization

**Project Goal:** Detect and classify road damage (Potholes, Cracks, Manholes) and provide actionable maintenance recommendations.

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random

# Set Paths
BASE_DIR = r"D:\Intern Project\Final Project\data"
IMAGE_DIR = os.path.join(BASE_DIR, "images")
LABEL_DIR = os.path.join(BASE_DIR, "labels")

# CONFIGURATION
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ['Pothole', 'Crack', 'Manhole']

### Step 1: Data Preparation
Loading images and labels, handling resizing, normalization, and augmentation.

In [ ]:
def load_dataset():
    image_paths = []
    labels = []
    
    for label_file in os.listdir(LABEL_DIR):
        if label_file.endswith('.txt'):
            img_name = label_file.replace('.txt', '.jpg')
            img_path = os.path.join(IMAGE_DIR, img_name)
            
            if os.path.exists(img_path):
                with open(os.path.join(LABEL_DIR, label_file), 'r') as f:
                    line = f.readline()
                    if line:
                        class_id = int(line.split()[0])
                        if class_id < 3:
                            image_paths.append(img_path)
                            labels.append(class_id)
    return np.array(image_paths), np.array(labels)

paths, y_data = load_dataset()
print(f"Dataset: {len(paths)} images across {len(CLASSES)} classes.")

#### handling Class Imbalance and Data Augmentation
Applying class weights to ensure the model doesn't drift toward the majority class.

In [ ]:
# Calculate Class Weights
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_data), y=y_data)
class_weights = dict(enumerate(weights))
print(f"Calculated Class Weights: {class_weights}")

# Data Augmentation Setup (Including Brightness and Blur as requested)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2], # Brightness adjustment
    horizontal_flip=True,
    validation_split=0.2
)

df = pd.DataFrame({'filename': paths, 'class': y_data.astype(str)})

train_gen = datagen.flow_from_dataframe(
    df, x_col='filename', y_col='class',
    target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_gen = datagen.flow_from_dataframe(
    df, x_col='filename', y_col='class',
    target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation', shuffle=False
)

### Step 2: Model Development
Building a Baseline CNN and then performing Transfer Learning with MobileNetV2.

In [ ]:
# 1. Baseline Model
def build_baseline():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 2. Transfer Learning (Recommended: MobileNetV2 for Lightweight deployment)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = True

# Fine-tune from this layer onwards (Fine-tuning selected layers)
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Step 3: Model Evaluation
Training the model and checking Precision, Recall, and Confusion Matrix.

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights
)

# Detailed Evaluation
y_true = val_gen.classes
y_pred = np.argmax(model.predict(val_gen), axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES)
plt.title('Confusion Matrix')
plt.show()

### Explainable AI: Grad-CAM Implementation
Visualizing which pixels influenced the model's decision.

In [ ]:
def generate_gradcam(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Save model for deployment
model.save("road_damage_final_model.h5")